In [ ]:
"""
    This is a script which utilizes the ObsInfo class.

    Create an instance of ObsInfo using data and parameters from a single Herschel
    observation. Compute the reference velocity for the spectral axis, 
    update FITS header keywords, and save to new FITS file.
"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard
import os
import numpy as np

# Local
from ObsInfo import ObsInfo
from dictionaryThings import loadDict

In [3]:
# Which program will be reading the output FITS file?
program = '3dBarolo'

topPath = '/Volumes/QbertPrimary/umdResearch/adapProposalNearby/'


# ----------------------------------------- #
# Necessary parameter file and dictionaries #
# ----------------------------------------- #
# Dictionary of galaxy properties
objDictName = topPath+'objectInfoDict.pkl'
# Dictionary of emission line properties.
lineDict = loadDict(topPath+'emiLineDict.pkl')
# Text file of line fitting parameters.
paramFileName = topPath + 'fittingParametersV4.txt'


# -------------------------------------------------- #
# Read in the parameter file containing line profile #
# velocity limits and continuum fitting information. #
# -------------------------------------------------- #
paramFileData = np.genfromtxt(paramFileName, dtype=None, autostrip=True,
                              names=True, encoding=None)

In [4]:
for x in range(len(paramFileData)):
    # ------------------------------------ #
    # Get the galaxy and line information. #
    # ------------------------------------ #
    obsInfo = ObsInfo(x, paramFileName, objDictName)
    
    # Name of the emission line.
    lineName = paramFileData['lineNameShort'][x]
    # Rest wavelength of the emission line.
    restWave = lineDict[lineName]['restWave']


    # ----------------------------------------- #
    # Create the base names for saving outputs. #
    # ----------------------------------------- #
    # Base for the object's file names.
    objectNameBase = (str(obsInfo.obsId)+'_'+obsInfo.objectName
                      +'_'+lineName)
    
    # Base path to the object's folders.
    objectPathBase = (topPath+'pySpecKitCube/maybeRun3/'+obsInfo.objectName
                          +'/1arc/'+program+'/')


    # -------------------------------------------------------------- #
    # Find the minimum velocity of the spectral axis.                #
    # This will be set as the reference velocity in the FITS header. #
    # -------------------------------------------------------------- #
    fitsFile = (topPath+'pySpecKitCube/'+obsInfo.objectName+'/1arc'
                        +'/instrCorrect/fits/'+objectNameBase
                        +'_clean_correct_contFlux.fits')

    if program == 'kpvslice':
        velCorr = True
    if program == '3dBarolo':
        velCorr = False

    obsInfo.findRefVel(fitsFile, restWave, velCorr=velCorr)


    # ----------------------------------------------------- #
    # Update/add spectral axis keywords in the FITS header. #
    # Also add beamsize information.                        #
    # ----------------------------------------------------- #
    fitsSavePath = objectPathBase + 'fits/'
    if (not os.path.exists(fitsSavePath)):os.makedirs(fitsSavePath)
    
    fitsSaveName = fitsSavePath+objectNameBase+'_hdrEditVel.fits'
    
    obsInfo.updateHdr(fitsFile, restWave, saveFile=fitsSaveName)


    break